In [1]:
import pretty_midi
import pandas as pd
import json
import glob
import os

In [2]:
master_dict = []

In [4]:
def extract_guitar_and_drums(midi_file):
    """This function extracts the guitar and drum tracks from a midi file.
       The input is a path to a midi file (for example: 'raw_data/song_name.mid') in string format
       The output is a dictionary with the song name, guitar track and drum track"""    
    
    mid = pretty_midi.PrettyMIDI(midi_file)
    
    guitars = []
    lengths_guitar = []
    drums = []
    lengths_drums = []
    
    for instrument in mid.instruments:
        if instrument.is_drum:
            drums.append(instrument)
            lengths_drums.append(len(instrument.notes))
        if (instrument.program >= 25) and (instrument.program <= 31):
            guitars.append(instrument)
            lengths_guitar.append(len(instrument.notes))
    drum_track = drums[lengths_drums.index(max(lengths_drums))]
    guitar_track = guitars[lengths_guitar.index(max(lengths_guitar))]
    song_title = os.path.splitext(os.path.basename(midi_file))[0]
    
    song_dict = {'title': song_title,
                 'down_beats': mid.get_downbeats(),
                 'guitar': guitar_track,
                 'drums': drum_track
                }
    
    return song_dict

In [161]:
def tracks_to_bars(song_dict: dict) -> dict:
    """This function accepts a dictionary as an input with 4 keys: 'title', 'down_beats', 'guitar', 'drums'.
    The function takes the guitar and drums, both pretty_midi instrument objects, and cuts them up into a sequence of individual bars.
    The output is a dictionary that contains the following keys/values: song_title, a list of guitar bars, a list of drum bars, and a list of the song's downbeats
    """
    
    new_dict={}
    new_dict['song_title']=song_dict['title']
    guitar = song_dict['guitar']
    drums = song_dict['drums']
    down_beats_array = song_dict['down_beats']


    guitar_bars_list = []
    drums_bars_list = []

    for index, start_time in enumerate(down_beats_array):
        if index < len(down_beats_array) - 1:
            end_time = down_beats_array[index+1] 
        else:
            end_time = down_beats_array[index] + 2
            
        guitar_bar = []
        drums_bar = []
        for guitar_note in guitar.notes:
            if (guitar_note.start >= start_time) and (guitar_note.end < end_time):
                guitar_bar.append(guitar_note)    

        for drum_note in drums.notes:
            if (drum_note.start >= start_time) and (drum_note.end < end_time):
                drums_bar.append(drum_note)
                  
        drums_bars_list.append(drums_bar)
        guitar_bars_list.append(guitar_bar)

    new_dict['guitar_bars'] = guitar_bars_list
    new_dict['drum_bars'] = drums_bars_list
    new_dict['down_beats'] = down_beats_array.tolist()
    return new_dict    

In [6]:
midi_paths = glob.glob('test_data/*.mid') + glob.glob('test_data/*/*.mid')

In [7]:
midi_paths

['test_data/Wish_You_Were_Here_1.mid',
 'test_data/Hotel_California_1.mid',
 'test_data/Zombie_5/Zombie_5.mid',
 'test_data/Zombie_4/Zombie_4.mid']

In [145]:
for path in midi_paths:
    song_dict = extract_guitar_and_drums(path)
    dict = tracks_to_bars(song_dict)

# js = json.dumps(dict, indent=4)
# # Open new json file if not exist it will create
# fp = open('song_bars_repo.json', 'a')
# # write to json file
# fp.write(js)
# # fp.write(',')
# # close the connection
# fp.close()

done
done
done
done


In [36]:
js = json.dumps(dict, indent=4)

# Open new json file if not exist it will create
fp = open('song_bars_repo.json', 'a')

# write to json file
fp.write(js)

# close the connection
fp.close()

In [50]:
def standardize_bars(list_of_bars, downbeats):
    """ 
    This function standardizes the timing of musical bars 
    so that each bar will start at the same time point.
    It gets a list of bars and the list of downbeats as inputs
    and returns a list of bars that all start with time = 1
    """
    
    for i in range(len(list_of_bars)):
        for j in range(len(list_of_bars[i])):
            if i == 0:
                list_of_bars[0][j].start = list_of_bars[0][j].start / downbeats[1]
                list_of_bars[0][j].end = list_of_bars[0][j].end / downbeats[1]

            list_of_bars[i][j].start = list_of_bars[i][j].start / downbeats[i]
            list_of_bars[i][j].end = list_of_bars[i][j].end / downbeats[i]

    return list_of_bars

In [141]:
def revert_standardization(list_of_std_bars: list, down_beats: list) -> list:
    """ 
    This function reverts the standardization performed on the timing of musical bars 
    so that each bar will start, once again, at their original timing.
    It gets a list of standardized bars and the list of original downbeats as inputs
    and returns a list of bars with their original timing.
    """
    original_down_beats = down_beats
    list_of_unstd_bars = []

    for i, std_bar in enumerate(list_of_std_bars):
        if std_bar == []:
            list_of_unstd_bars.append(std_bar)
        else:
            list_of_notes = []
            for note in std_bar:           
                if i == 0:
                    note.start = note.start*original_down_beats[1]
                    note.end = note.end*original_down_beats[1]
                    list_of_notes.append(note)
                
                note.start = note.start*original_down_beats[i]
                note.end = note.end*original_down_beats[i]
                list_of_notes.append(note)
            list_of_unstd_bars.append(list_of_notes)

    return list_of_unstd_bars

In [153]:
unstd_bars = revert_standardization(std_bars, dict['down_beats'])

In [155]:
dict['drum_bars']

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [Note(start=11.636352, end=11.706049, pitch=46, velocity=127),
  Note(start=11.999988, end=12.075746, pitch=38, velocity=127),
  Note(start=12.363624, end=12.442412, pitch=38, velocity=115),
  Note(start=12.545442, end=12.618169, pitch=38, velocity=118),
  Note(start=12.727260, end=12.815139, pitch=38, velocity=120),
  Note(start=12.909078, end=12.990896, pitch=38, velocity=127)],
 [Note(start=13.090896, end=13.178775, pitch=55, velocity=127),
  Note(start=13.454532, end=13.512108, pitch=46, velocity=112),
  Note(start=13.090896, end=13.530290, pitch=36, velocity=127),
  Note(start=13.818168, end=13.896956, pitch=46, velocity=112),
  Note(start=13.818168, end=14.227258, pitch=38, velocity=127),
  Note(start=14.181804, end=14.254531, pitch=46, velocity=112)],
 [Note(start=14.545440, end=14.636349, pitch=46, velocity=112),
  Note(start=14.545440, end=14.715137, pitch=36, velocity=127),
  Note(start=14.909076, end=15.012106, pitch=46, velocity=112)

In [154]:
unstd_bars == dict['drum_bars']

True

In [152]:
unstd_bars

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [Note(start=1.000000, end=1.005990, pitch=46, velocity=127),
  Note(start=1.031250, end=1.037760, pitch=38, velocity=127),
  Note(start=1.062500, end=1.069271, pitch=38, velocity=115),
  Note(start=1.078125, end=1.084375, pitch=38, velocity=118),
  Note(start=1.093750, end=1.101302, pitch=38, velocity=120),
  Note(start=1.109375, end=1.116406, pitch=38, velocity=127)],
 [Note(start=1.000000, end=1.006713, pitch=55, velocity=127),
  Note(start=1.027778, end=1.032176, pitch=46, velocity=112),
  Note(start=1.000000, end=1.033565, pitch=36, velocity=127),
  Note(start=1.055556, end=1.061574, pitch=46, velocity=112),
  Note(start=1.055556, end=1.086806, pitch=38, velocity=127),
  Note(start=1.083333, end=1.088889, pitch=46, velocity=112)],
 [Note(start=1.000000, end=1.006250, pitch=46, velocity=112),
  Note(start=1.000000, end=1.011667, pitch=36, velocity=127),
  Note(start=1.025000, end=1.032083, pitch=46, velocity=112),
  Note(start=1.025000, end=1

In [150]:
std_bars = standardize_bars(dict['drum_bars'], dict['down_beats'])

In [151]:
std_bars

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [Note(start=1.000000, end=1.005990, pitch=46, velocity=127),
  Note(start=1.031250, end=1.037760, pitch=38, velocity=127),
  Note(start=1.062500, end=1.069271, pitch=38, velocity=115),
  Note(start=1.078125, end=1.084375, pitch=38, velocity=118),
  Note(start=1.093750, end=1.101302, pitch=38, velocity=120),
  Note(start=1.109375, end=1.116406, pitch=38, velocity=127)],
 [Note(start=1.000000, end=1.006713, pitch=55, velocity=127),
  Note(start=1.027778, end=1.032176, pitch=46, velocity=112),
  Note(start=1.000000, end=1.033565, pitch=36, velocity=127),
  Note(start=1.055556, end=1.061574, pitch=46, velocity=112),
  Note(start=1.055556, end=1.086806, pitch=38, velocity=127),
  Note(start=1.083333, end=1.088889, pitch=46, velocity=112)],
 [Note(start=1.000000, end=1.006250, pitch=46, velocity=112),
  Note(start=1.000000, end=1.011667, pitch=36, velocity=127),
  Note(start=1.025000, end=1.032083, pitch=46, velocity=112),
  Note(start=1.025000, end=1